In [1]:
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import lil_matrix
import matplotlib.pyplot as plt
import pandas as pd
import math
import pickle
import time
import datetime
import random
from pprint import pprint

from SVD import Metrics, SVDPredictor
from RecData import RecData

In [2]:
recs = pd.read_csv('data/pruned.csv')
USED_COLS = ['app_id', 'is_recommended', 'user_id']
recs = recs[USED_COLS]

item_data = pd.read_csv('data/games.csv')
titles = item_data[['app_id', 'title']]

recs.head()

,app_id,is_recommended,user_id
0,534380,True,10531
1,42700,False,185293
2,602960,True,319249
3,976730,False,747545
4,1091500,True,2113544


In [3]:
random.seed(42)
np.random.seed(42)

rec_data = RecData()
rec_data.create_from_dataframe(recs)
rec_data.set_titles(titles)

train_data, test = rec_data.leave_k_out_split()
train_data, val = train_data.leave_k_out_split()

In [ ]:
EPOCHS = 20

random.seed(42)
np.random.seed(42)

# Fit and predict with svd
svd_predictor = SVDPredictor(
    train_data.get_num_users(), train_data.get_num_items(), k=100, epochs=EPOCHS, 
    learning_rate=0.005, C=0.02)
svd_predictor.fit(train_data.get_matrix(), validation_set=val)

# Plot error
errors = svd_predictor.get_train_errors()
val_errors = svd_predictor.get_val_errors()
plt.plot(range(len(errors)), errors, label='Training error')
plt.plot(range(len(val_errors)), val_errors, label='Test error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.title('Error vs Epoch')
plt.legend()
plt.show()

# Compute rmse
predictions = svd_predictor.predict([(user, item) for user, item, _ in val])
predictions = [prediction + (val[i][2],) for i, prediction in enumerate(predictions)]
metrics = Metrics()
rmse = metrics.rmse(predictions)
print("Test rmse", rmse)

# Ensure file exists
model_dir = model_dir = "saved_models/{}-{}-{}-{}-{}-{}_{}".format(
    *(time.localtime()[:6] + (round(rmse, 4), ))).replace(".", "-") + ".pkl" 
file = open(model_dir, 'a')
file.close()

# Save model
print("Saving model...")
with open(model_dir, 'wb') as file:
    pickle.dump([svd_predictor, rec_data, train_data, val, test], file)
print("Done saving model.")

In [ ]:
# Test top-n list for user 0
top_n = svd_predictor.top_n(0)
print("Predicted top n:")
pprint([(r, train_data.index_to_title(index)) for r, index in top_n])

# User 0 true top n list
print("True top n:")
true_top = rec_data.top_n(0, 100)
pprint([rec_data.index_to_title(index) for _, index in true_top])

In [ ]:
rec_data.search_title('1330')

In [ ]:
prefs = [(606, 1), (1628, 1), (391, 0), (1330, 0), (500, 0), (709, 1), (840, 0), 
         (58, 1), (456, 1), (153, 0), (158, 0), (1160, 0), (761, 0),
                              (1771, 0), (544, 0), (312, 0), (277, 0)]
pprint([(index, rec_data.index_to_title(index), pref) for index, pref in prefs])
prefs = rec_data.create_prefs(prefs)    

In [20]:
svd_predictor._partial_batch_size = int(0)
start_time = time.time()
svd_predictor.partial_fit(prefs)
print("Completed partial fit in", time.time() - start_time, "seconds")

Epoch 0/Training error: 0.3335227855732511/Time: 3.32 seconds
Epoch 1/Training error: 0.3335216319019268/Time: 3.33 seconds
Epoch 2/Training error: 0.33352083110605046/Time: 3.27 seconds
Epoch 3/Training error: 0.3335202533750534/Time: 3.26 seconds
Epoch 4/Training error: 0.33351985188113237/Time: 3.24 seconds
Completed partial fit in 16.47112774848938 seconds
Predicted top n
[(1.7077738155050715, 'Hexcells Infinite'),
 (1.6895720591944543, 'Lil Gator Game'),
 (1.688066401769133, 'The Case of the Golden Idol'),
 (1.687407148980516, 'Turing Complete'),
 (1.6819600088797104, 'Dungeon Warfare 2'),
 (1.6792200525321865, 'Thief™ II: The Metal Age'),
 (1.6781844619468393, 'Baba Is You'),
 (1.6767923925314212, 'A Hat in Time'),
 (1.6742712744118848, 'Later Alligator'),
 (1.6722390027958658, 'Incredibox')]


In [21]:
top_n = svd_predictor.top_n(train_data.get_matrix().shape[0], 20)
print("Predicted top n")
pprint([(r, train_data.index_to_title(index)) for r, index in top_n])

Predicted top n
[(1.7077738155050715, 'Hexcells Infinite'),
 (1.6895720591944543, 'Lil Gator Game'),
 (1.688066401769133, 'The Case of the Golden Idol'),
 (1.687407148980516, 'Turing Complete'),
 (1.6819600088797104, 'Dungeon Warfare 2'),
 (1.6792200525321865, 'Thief™ II: The Metal Age'),
 (1.6781844619468393, 'Baba Is You'),
 (1.6767923925314212, 'A Hat in Time'),
 (1.6742712744118848, 'Later Alligator'),
 (1.6722390027958658, 'Incredibox'),
 (1.6716464168677994, 'Monolith'),
 (1.6707495581588674, 'Touhou Tenkuushou ~ Hidden Star in Four Seasons.'),
 (1.6706483404688854, 'West of Loathing'),
 (1.6686661356641215, 'Epistory - Typing Chronicles'),
 (1.6662410650293438, 'Cruelty Squad'),
 (1.6661304650749214, 'Vampire: The Masquerade - Bloodlines'),
 (1.6654861746299652, 'POSTAL 2'),
 (1.6653458008541528, 'Enigmatis 2: The Mists of Ravenwood'),
 (1.6647655953700902, 'Bastion'),
 (1.6641237197117376, 'Walkabout Mini Golf VR')]
